In [1]:
from app.main.Main import Main

app = Main()
app.code = "VT" # 股票代碼
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

VT
2013-01-01-2023-01-01
final_profit_rate: 41.84%
final_profit_rate_with_dividend: 41.84%


In [2]:
from app.main.Main import Main

app = Main()
app.code = "ACDD01" # 安聯台灣大壩基金-A累積型(台幣)
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = True
app.using_taiwan_stock_crawl = False
app.using_taiwan_etf_crawl = False

app.launch()

ACDD01
2013-01-01-2023-01-01
final_profit_rate: 131.36%
final_profit_rate_with_dividend: 131.36%


In [3]:
from app.main.Main import Main

app = Main()
app.code = "0050.TW" # 股票代碼
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

0050.TW
2013-01-01-2023-01-01
final_profit_rate: 40.84%
final_profit_rate_with_dividend: 81.8%


In [4]:
from app.main.Main import Main

app = Main()
app.code = "0056.TW" # 股票代碼
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

0056.TW
2013-01-01-2023-01-01
final_profit_rate: -1.73%
final_profit_rate_with_dividend: 49.39%


In [5]:
from app.main.Main import Main

app = Main()
app.code = "2884.TW" #玉山金
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

2884.TW
2013-01-01-2023-01-01
final_profit_rate: 100.9%
final_profit_rate_with_dividend: 229.2%


In [6]:
from app.main.Main import Main

app = Main()
app.code = "5880.TW" # 合庫金
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

5880.TW
2013-01-01-2023-01-01
final_profit_rate: 121.84%
final_profit_rate_with_dividend: 291.19%


In [7]:
from app.main.Main import Main

app = Main()
app.code = "9941.TW" # 裕融
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

9941.TW
2013-01-01-2023-01-01
final_profit_rate: 230.02%
final_profit_rate_with_dividend: 1400.41%


In [8]:
from app.main.Main import Main

app = Main()
app.code = "2882.TW" # 國泰金
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

2882.TW
2013-01-01-2023-01-01
final_profit_rate: 16.29%
final_profit_rate_with_dividend: 98.16%


In [9]:
from app.main.Main import Main

app = Main()
app.code = "2330.TW" # 台積電
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

2330.TW
2013-01-01-2023-01-01
final_profit_rate: 195.65%
final_profit_rate_with_dividend: 394.51%


In [10]:
from app.main.Main import Main

app = Main()
app.code = "2480.TW" # 敦陽科
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

2480.TW
2013-01-01-2023-01-01
final_profit_rate: 235.8%
final_profit_rate_with_dividend: 4692.99%


In [11]:
from app.main.Main import Main

app = Main()
app.code = "2618.TW" # 長榮航空
app.start_date = "2013-01-01" # 起始日期
app.end_date = "2023-01-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

app.launch()

2618.TW
2013-01-01-2023-01-01
final_profit_rate: 106.31%
final_profit_rate_with_dividend: 136.53%


In [12]:
from app.main.Main import Main

app = Main()
app.start_date = "2022-05-01" # 起始日期
app.end_date = "2023-05-01" # 結束日期
app.investment_amount = 1000
app.purchase_day = 1
app.is_fund = False
app.using_taiwan_stock_crawl = True
app.using_taiwan_etf_crawl = False

code_list =  [
    "0050.TW", # 股票代碼
    "2480.TW", # 敦陽科
    "9941.TW", # 裕融
]
# code_list =  [
#     "0056.TW", # 股票代碼
#     "0050.TW", # 股票代碼
# ]
# code_list =  [
#     "0050.TW", # 股票代碼
#     "2881.TW", # 富邦金
#     "2882.TW", # 國泰金
#     "2884.TW", # 玉山金
#     "2887.TW", # 台新金
#     "5880.TW", # 合庫金
# ]
for code in code_list:
    app.code = code
    app.launch()

0050.TW
2022-05-01-2023-05-01
final_profit_rate: 1.93%
final_profit_rate_with_dividend: 4.07%
2480.TW
2022-05-01-2023-05-01
final_profit_rate: 18.9%
final_profit_rate_with_dividend: 18.9%
9941.TW
2022-05-01-2023-05-01
final_profit_rate: 12.24%
final_profit_rate_with_dividend: 12.24%
